In [7]:
import pickle

with open('plants_dic.pkl','rb') as f:
    plants_dic = pickle.load(f)

flowers = list(plants_dic['flower'])

with open('flowers_list.pkl','wb') as f:
    pickle.dump(flowers,f)

In [8]:
with open('flowers_list.pkl','rb') as f:
    flowers_list = pickle.load(f)

flowers_list

['ヒメヅタ',
 'ハイブリッドジギタリス（スーパージギタリス）',
 'プリムラマラコイデス',
 'フジバカマ(藤袴)',
 'エラチオールベゴニア(リーガースベゴニア)',
 'ネコノヒゲ',
 'クレマチス・ジャックマニー系',
 'ヤセウツボ',
 'クレマチス・フォステリー系',
 'セルリア',
 'キンラン（金蘭）',
 'ギンラン（銀蘭）',
 'ムラサキケマン',
 'ペンタス',
 'ハゲイトウ(葉鶏頭)',
 'モノプシス',
 'ブルーデージー',
 'カスミソウ(かすみそう・かすみ草)',
 'オキナグサ(翁草)',
 'イワタバコ',
 'ブルースター（オキシペタラム）',
 'コスモス(秋桜)',
 'シレネ・ユニフローラ',
 'チョウジソウ（丁字草）',
 '薄（ススキ）',
 'カリブラコア',
 'トケイソウ(時計草)',
 'ハマダイコン',
 'Cyclamen Creator Yasuhiro Takahashi(CCYT) 江戸ノ青',
 'リナム（フラックス）',
 'オリエンタルポピー',
 'ジャスミン',
 'ヒメツルソバ',
 'Cyclamen Creator Yasuhiro Takahashi(CCYT) 瑠璃玉',
 'マーガレット',
 'Cyclamen Creator Yasuhiro Takahashi(CCYT) 胡蝶',
 'モナルダ・ベルガモ',
 'ダチュラ(朝鮮朝顔)',
 'グレコマ',
 'サクシセラ・フロステッドパールズ',
 'アスター',
 'クライミング・ローズ(つるバラ)',
 'シャガ',
 'カタバミ',
 'セイタカアワダチソウ',
 'フォーチュンベゴニア',
 'ヒポエステス',
 'ユリオプスデージー',
 '茉莉花（マツリカ）',
 'アザレア',
 'アスチルベ',
 'シロタエギク',
 'クレマチス・フロリダ系',
 'ベロニカ・オックスフォードブルー',
 '雪の下（ユキノシタ）',
 'アルケミラモリス',
 'ラスティセージ',
 'ペラルゴニウム・パールグレイ',
 'バーベナ',
 '半夏生（ハンゲショウ）',
 'プルモナリア',
 'アイリス',
 'Cyclamen Creator Yasuhiro Takahashi(CCYT) 

In [12]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import time
import datetime
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
import pickle

# 拡張子を取得
def get_extension(url):
    url_lower = url.lower()
    for img_ext in IMG_EXTS:
        if img_ext in url_lower:
            extension = '.jpg' if img_ext == '.jpeg' else img_ext
            break
    else:
        extension = ''
    return extension

# urlの画像を取得しファイルへ書き込む
def download_image(url, path, loop):
    result = False
    for i in range(loop):
        try:
            r = requests.get(url, headers=HTTP_HEADERS, stream=True, timeout=10)
            r.raise_for_status()
            with open(path, 'wb') as f:
                f.write(r.content)
        except requests.exceptions.SSLError:
            print('***** SSL エラー')
            break  # リトライしない
        except requests.exceptions.RequestException as e:
            print(f'***** requests エラー({e}): {i + 1}/{RETRY_NUM}')
            time.sleep(1)
        else:
            result = True
            break  # try成功
    return result


with open('flowers_list.pkl','rb') as f:
    flowers_list=pickle.load(f)


# for category,plants_set in plants_dic.items():
#     plants_dic[category]=list(plants_set)
 
# print(plants_dic)

tm_start = time.time()  # 処理時間計測用
dt_now = datetime.datetime.now()  # 現在日時
dt_date_str = dt_now.strftime('%Y/%m/%d %H:%M')
print(dt_date_str)
 
# QUERY = 'アイビーゼラニウム　花　葉'  # 検索ワード
LIMIT_DL_NUM = 300               # ダウンロード数の上限
# SAVE_DIR = 'output_scraping/test01'  # 出力フォルダへのパス（フォルダがない場合は自動生成する）
# SAVE_DIR = QUERY  # 出力フォルダへのパス（フォルダがない場合は自動生成する）
FILE_NAME = ''                       # ファイル名（ファイル名の後ろに０からの連番と拡張子が付く）
TIMEOUT = 60                     # 要素検索のタイムアウト（秒）
ACCESS_WAIT = 1                  # アクセスする間隔（秒）
RETRY_NUM = 3                    # リトライ回数（クリック、requests）
DRIVER_PATH = './chromedriver'        # chromedriver.exeへのパス
 
# Chromeをヘッドレスモードで起動
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--start-maximized')
options.add_argument('--start-fullscreen')
options.add_argument('--disable-plugins')
options.add_argument('--disable-extensions')
driver = webdriver.Chrome(DRIVER_PATH, options=options)
 
# タイムアウト設定
driver.implicitly_wait(TIMEOUT)
 
tm_driver = time.time()
print('WebDriver起動完了', f'{tm_driver - tm_start:.1f}s')

# for category,plants_set in plants_dic.items():
for flower in flowers_list:
    QUERY=flower+' 花'
    SAVE_DIR = 'plants/flower/'+flower  # 出力フォルダへのパス（フォルダがない場合は自動生成する）

    # Google画像検索ページを取得
    url = f'https://www.google.com/search?q={QUERY}&tbm=isch'
    driver.get(url)
    
    tm_geturl = time.time()
    print('Google画像検索ページ取得', f'{tm_geturl - tm_driver:.1f}s')
    print(flower)
    
    tmb_elems = driver.find_elements_by_css_selector('#islmp img')
    tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
    
    count = len(tmb_alts) - tmb_alts.count('')
    print(count)
    
    while count < LIMIT_DL_NUM:
        prev_count = count
        # ページの一番下へスクロールして新しいサムネイル画像を表示させる
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
    
        # サムネイル画像取得
        tmb_elems = driver.find_elements_by_css_selector('#islmp img')
        tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
    
        count = len(tmb_alts) - tmb_alts.count('')
        print(count)
        if count == prev_count:
            break
    
    # サムネイル画像をクリックすると表示される領域を取得
    imgframe_elem = driver.find_element_by_id('islsp')
    
    # 出力フォルダ作成
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    # HTTPヘッダ作成
    HTTP_HEADERS = {'User-Agent': driver.execute_script('return navigator.userAgent;')}
    print(HTTP_HEADERS)           
                
    # ダウンロード対象のファイル拡張子
    IMG_EXTS = ('.jpg', '.jpeg', '.png')
    
    tm_thumbnails = time.time()
    print('サムネイル画像取得', f'{tm_thumbnails - tm_geturl:.1f}s')
    
    # ダウンロード
    EXCLUSION_URL = 'https://lh3.googleusercontent.com/'  # 除外対象url
    count = 0
    url_list = []
    for tmb_elem, tmb_alt in zip(tmb_elems, tmb_alts):
        
        if tmb_alt == '':
            continue
    
        print(f'{count}: {tmb_alt}')
    
        for i in range(RETRY_NUM):
            try:
                # サムネイル画像をクリック
                tmb_elem.click()
            except ElementClickInterceptedException:
                print(f'***** click エラー: {i + 1}/{RETRY_NUM}')
                driver.execute_script('arguments[0].scrollIntoView(true);', tmb_elem)
                time.sleep(1)
            else:
                break  # try成功
        else:
            print('***** キャンセル')
            continue  # リトライ失敗
            
        # アクセス負荷軽減用のウェイト
        time.sleep(ACCESS_WAIT)
        
        alt = tmb_alt.replace("'", "\\'")
        try:
            img_elem = imgframe_elem.find_element_by_css_selector(f'img[alt=\'{alt}\']')
        except NoSuchElementException:
            print('***** img要素検索エラー')
            print('***** キャンセル')
            continue
    
        # url取得
        tmb_url = tmb_elem.get_attribute('src')  # サムネイル画像のsrc属性値
    
        for i in range(RETRY_NUM):
            url = img_elem.get_attribute('src')
            if EXCLUSION_URL in url:
                print('***** 除外対象url')
                url = ''
                break
            elif url == tmb_url:  # src属性値が遷移するまでリトライ
                print(f'***** urlチェック: {i + 1}/{RETRY_NUM}')
    #             print(f'***** {url}')
                time.sleep(1)
                url = ''
            else:
                break
    
        if url == '':
            print('***** キャンセル')
            continue
    
    #     print(f'url: {url}')
    
        # 画像を取得しファイルへ保存
        ext = get_extension(url)
        if ext == '':
            print(f'***** urlに拡張子が含まれていないのでキャンセル')
            # print(f'{url}')
            continue
    
        filename = f'{FILE_NAME}{count}{ext}'
        path = SAVE_DIR + '/' + filename
        result = download_image(url, path, RETRY_NUM)
        if result == False:
            print('***** キャンセル')
            continue
        url_list.append(f'{filename}: {url}')
    
        # ダウンロード数の更新と終了判定
        count += 1
    #    print(f'\r{count}/{LIMIT_DL_NUM}', end='')  # 進捗表示
        if count >= LIMIT_DL_NUM:
    #        time.sleep(1)  # 進捗表示ウェイト
    #        print(f'\r{" " * 7}\r', end='')  # 進捗非表示        
            break
    
    tm_end = time.time()
    print('ダウンロード', f'{tm_end - tm_thumbnails:.1f}s')
    print('------------------------------------')
    total = tm_end - tm_start
    total_str = f'トータル時間: {total:.1f}s({total/60:.2f}min)'
    count_str = f'ダウンロード数: {count}'
    print(total_str)
    print(count_str)
    
    # urlをファイルへ保存
    # path = SAVE_DIR + '/' + '_url.txt'
    # with open(path, 'w', encoding='utf-8') as f:
    #     f.write(dt_date_str + '\n')
    #     f.write(total_str + '\n')
    #     f.write(count_str + '\n')
    #     f.write('\n'.join(url_list))
 
driver.quit()

2021/05/30 20:37
WebDriver起動完了 1.6s
Google画像検索ページ取得 1.7s
ヒメヅタ
50
60
60
{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/90.0.4430.212 Safari/537.36'}
サムネイル画像取得 2.8s
0: ヒメヅタの育て方 | LOVEGREEN(ラブグリーン)
1: 楽天市場】ヒメヅタ：GardenShop LaCotto
2: 希少種つる性植物 『ヒメヅタ』 | すべての商品 | | Junk sweet Garden tef*tef*
3: ヒメヅタの投稿画像 by kusakiさん｜アジサイとあじさいコンテストとグリーンギャラリーガーデンズと寄せ植えと多肉じゃない寄せ植え  (2017月6月14日)｜🍀GreenSnap（グリーンスナップ）
4: ヒメヅタ １７０６１５ | 樹木
5: ヒメヅタの育て方 | LOVEGREEN(ラブグリーン)
6: ヒメヅタの投稿画像一覧｜🍀GreenSnap（グリーンスナップ）
7: ヒメヅタのカゴ植え : はなわっか
8: ヒメヅタの投稿画像 by あやぽんさん｜グリーンのある暮らしとなつと今日も元気!!とmyfavoriteとナチュラルスタイルとバルコニスト  (2017月8月10日)｜🍀GreenSnap（グリーンスナップ）
9: ヒメヅタ - 植物
***** urlチェック: 1/3
***** urlチェック: 2/3
***** urlチェック: 3/3
***** キャンセル
9: 樹木売り場へＧｏ！ : kusakiの「この植物をお買い!」
10: 蔓植物を植えましょう！ | 河野自然園社長 井上まゆ美のお仕事日和２
11: 希少種つる性植物 『ヒメヅタ』 | すべての商品 | | Junk sweet Garden tef*tef*
12: ヒメヅタの葉っぱが展開してきました、 ...「花便り🌼2018」のアルバム - みんなの趣味の園芸 id:602654
13: ツル性植物はお庭の秘密兵器だとか・・・ : フラワーガーデン花音
14: 

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=50031): Max retries exceeded with url: /session/181e91cc00c58e80125f23be949f0d94/element/e856b9eb-d2c4-493c-acd3-1b38f1364bda/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x112200c40>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [6]:
import pickle
import os

flowers={dirname for dirname in os.listdir('./plants/flower') if not dirname.startswith('.')}
selected_flowers={dirname for dirname in os.listdir('./plants/flower/.selected_flowers/') if not dirname.startswith('.')}
collected_flowers=flowers|selected_flowers

with open('plants_dic.pkl','rb') as f:
    plants_dic=pickle.load(f)

plants_dic['flower']=plants_dic['flower']-collected_flowers
plants_dic['flower']=plants_dic['flower']-{
    'Cyclamen Creator Yasuhiro Takahashi(CCYT) 冬化粧',
    'Cyclamen Creator Yasuhiro Takahashi(CCYT) 月下',
    'Cyclamen Creator Yasuhiro Takahashi(CCYT) 江戸ノ青',
    'Cyclamen Creator Yasuhiro Takahashi(CCYT) 瑠璃玉',
    'Cyclamen Creator Yasuhiro Takahashi(CCYT) 胡蝶',
    'アネモネ・バージニアナ',
    'シンニンギア・バター＆クリーム',
    'シンニンギア・ラブリー',
    'スミレイワギリソウ（菫岩桐草）',
    'セレナーディア ビクトリアブルー',
    'トラディスカンティア・クラッシフォリア・アカウリス',
    'ナズナ（タラスピ）',
    'フッチンシア',
    'ポテンティラ・スルフレア（サルフレア）',
    'ルエリア・ブラックビューティー',
    'ルリオダマキ',
    'ワイルドオーツ（グリーンスケール）',}

plants_dic['flower']=sorted(plants_dic['flower'])

plants_dic

{'flower': ['水芭蕉（ミズバショウ）',
  '皇帝ダリア',
  '矢車菊(ヤグルマギク)',
  '筑紫カラマツ',
  '芝桜(シバザクラ)',
  '薄（ススキ）'],
 'glass': {'カレックス',
  'コウライシバ(高麗芝)',
  'ニューサイラン(マオラン)',
  'パニカム',
  'パンパスグラス',
  'フェザーグラス',
  'ペニセツム（ペニセタム）',
  'ヤブラン',
  'リュウノヒゲ（ジャノヒゲ）',
  '猫草(ネコグサ)'},
 'bulb': {'サフラン',
  'スノーフレーク',
  'チューリップ',
  'ヒヤシンス',
  'ラナンキュラス',
  '原種チューリップ',
  '水仙（スイセン）'},
 'herb': {'アップルミント',
  'アメジストセージ（サルビア・レウカンサ）',
  'アルカネット',
  'アルケミラモリス',
  'アロマティカス',
  'イタリアンパセリ',
  'イングリッシュラベンダー',
  'エルダーフラワー',
  'オレガノ',
  'オーデコロンミント',
  'カモミール(カミツレ)',
  'カレープラント',
  'クリーピングタイム',
  'クリーピングボリジ',
  'コモンセージ',
  'コモンタイム',
  'サラダバーネット',
  'サントリナ',
  'ジャーマンカモミール',
  'スイートバジル',
  'スペアミント',
  'セルバチコ',
  'セントランサス',
  'セージ',
  'タイム',
  'チェリーセージ',
  'チャイブ',
  'チャービル',
  'ディル',
  'ニホンハッカ',
  'バジル',
  'バタフライピー（チョウマメ、クリトリア）',
  'パイナップルミント',
  'パクチー(コリアンダー)',
  'パープルセージ',
  'フェンネル',
  'フレンチラベンダー',
  'ブッシュバジル',
  'ヘリオトロープ',
  'ペインテッドセージ（サルビアホルミナム）',
  'ペニーロイヤルミント',
  'ペパーミント',
  'ホワイトセージ',
  'ホーリーバジル(トゥルシー)',
  'ボリジ(ルリジサ)',
  'ミント',
  'モナルダ